# DFT Manager
---
### Type in the name of your project, then click "Add Project" to create it. If it doesn't appear in the directory, reload the page and it should.

### To remove a project, select a project entry in the directory or type its name in the name box. Then, click "Remove Project" to remove the project and all its associated data.

# DEV NOTE:  Run all cells and the directory will appear at the bottom

In [1]:
import shutil
import uuid
import time
import getpass
import socket

import pathlib as pl

import qgrid

import pandas as pd

from ipywidgets import Button, Text, HBox
from IPython.display import clear_output, display
from tinydb import *

qgrid.enable()

In [2]:
INSTALL_PATH = str(pl.Path.cwd())  # determine where dftman is installed

GLOBAL_LIB_PATH = '/apps/dftman/dev/lib'
LOCAL_LIB_PATH = 'lib'

GLOBAL_BIN_PATH = '/apps/dftman/dev/bin'

LOCAL_SRC_PATH = 'src'

DB_PATH = 'dftman.json'

PROJECTS_PATH = 'projects'
PROJECTS_TABLE = 'projects'

DEFAULT_TOOL = 'default.ipynb'

In [3]:
def install_libs():
    '''
    Install all available versions of dftman librar(y/ies)
    Currently copies library files from global LIB_PATH to
        the local 'lib' directory
    '''
    # install / update library files
    if pl.Path(LOCAL_SRC_PATH).exists():  # src is only in dev, lib already present
        print('Not installing libs, already in dev!')
        return
    
    locallib = pl.Path(LOCAL_LIB_PATH)
    globallib = pl.Path(GLOBAL_LIB_PATH)
    
    # create local lib directory if necessary
    if not locallib.exists():
        locallib.mkdir()
    
    # scan globallib and add any missing softlinks
    for f in globallib.iterdir():
        dst = locallib / f.name
        if not dst in locallib.iterdir():
            dst.symlink_to(f)

def add_project(_):
    '''
    Add a new project by adding a project entry to the database,
        creating the project directory, and copying the DEFAULT_TOOL
        to the project directory
    :param name_field: Jupyter widget for name insertion
    '''
    name = name_field.value
    # uuid = uuid.uuid4().hex  # not sure we need a uuid for the projects
    
    project_path = pl.Path(PROJECTS_PATH) / '{}'.format(name)
    tool_source = pl.Path(GLOBAL_BIN_PATH) / DEFAULT_TOOL
    tool_dest = project_path / '{}.ipynb'.format(name)
    
    tool_link = '<a href="{}" target="_blank">Installed Tool</a>'.format(tool_dest)
    
    # load database and projects table
    db = TinyDB(DB_PATH)
    projects_table = db.table('projects')
    
    # add project entry to projects table
    projects_table.insert({
        'name': str(name),
        'path': str(project_path),
        'link': str(tool_link),
        'creation_time': time.asctime(time.gmtime()),  # UTC creation time
        'creation_user': getpass.getuser(),
        # 'creation_host': socket.gethostname()
    })
        
    # make directories
    project_path.mkdir(parents=True)
    # copy files
    # print('copying {} -> {}'.format(tool_source, tool_dest))
    shutil.copy(str(tool_source), str(tool_dest))
    # show projects
    show_projects()
    
def delete_project(_):
    '''
    Delete a project by adding finding and deleting the project from
        the database and removing the project directory tree
    :param name_field: Jupyter widget for name insertion
    '''
    name = name_field.value
    # load database and projects table
    db = TinyDB(DB_PATH)
    projects_table = db.table('projects')
    # query for project
    query = Query()
    project = projects_table.get(query.name == name)
    doc_id = project.doc_id
    # remove project directory
    rm_path = str(pl.Path.cwd() / project['path'])
    shutil.rmtree(rm_path)
    # remove project document
    projects_table.remove(doc_ids=[doc_id])
    # purge project table
    db.purge_table(name)
    
    show_projects()
    return

In [4]:
add_button = Button(
                description='Add Project',
                icon='plus',
                tooltip='Add a New Project',
                button_style='success',
                disabled=True
            )

delete_button = Button(
                description='Delete Project',
                icon='minus',
                tooltip='Delete Project and its Files',
                button_style='danger',
                disabled=True
            )
style = {'description_width': 'initial'}
name_field = Text(description='Project Name', style=style)

In [5]:
def project_name(change):
    '''
    Callback when the name_field changes
    When the name changes, should check if the project
        exists and enable / disable Add / Delete buttons
        as necessary
    :param change: ipywidget state change dictionary
    '''
    name = change['new']
    
    # load database and projects table
    db = TinyDB(DB_PATH)
    projects_table = db.table('projects')
    # query for project
    query = Query()
    result = projects_table.search(query.name == name)
    
    if result:
        add_button.disabled = True
        delete_button.disabled = False
    else:
        add_button.disabled = False
        delete_button.disabled = True

def select_cb(event, w):
    '''
    Callback when a different row of the qgrid is selected
    This will change the name in field_name to the appropriate
        name in the table, and project_name will handle the rest
    :param event: qgrid state change event dictionary
    :w: qgrid status information dictionary
    '''
    ind = event['new'][0]
    # load database and projects table
    db = TinyDB(DB_PATH)
    projects_table = db.table('projects') 
    projects = projects_table.all()
    df = pd.DataFrame(projects).set_index('name')
    # find name
    name = df.iloc[ind].name
    name_field.value = name    
    
def show_projects():
    '''
    Show the projects and their metadata as listed in the
        database using a qgrid grid widget
    '''
    clear_output()
    # load database and projects table
    db = TinyDB(DB_PATH)
    projects_table = db.table('projects')
    projects = projects_table.all()
    # display message if no projects
    if len(projects) == 0:
        print('No projects exist. Make a project first.')
        display(qgrid.show_grid(pd.DataFrame([]), grid_options={'editable': False}))
    else:
        df = pd.DataFrame(projects).set_index('name')
        display(qgrid.show_grid(df, grid_options={'editable': False}))
        qgrid.on('selection_changed', select_cb)

    # add_button.on_click(callback=add_project)
    add_button.on_click(callback=add_project)
    delete_button.on_click(callback=delete_project)
    
    name_field.value = ''
    name_field.observe(project_name, names='value')
        
    display(HBox([name_field, add_button, delete_button]))
    
    return

In [6]:
install_libs()

Not installing libs, already in dev!


In [7]:
show_projects()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': False, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, id='63324267-750b-4afe-b7bc-5e30d5191af7', precision=5)

HBox(children=(Text(value='', description='Project Name', style=DescriptionStyle(description_width='initial')), Button(button_style='success', description='Add Project', icon='plus', style=ButtonStyle(), tooltip='Add a New Project'), Button(button_style='danger', description='Delete Project', disabled=True, icon='minus', style=ButtonStyle(), tooltip='Delete Project and its Files')))